In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score

In [2]:
df = pd.read_csv('mental-heath-in-tech-2016_20161114.csv')
df.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,6-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,6-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,6-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes


In [3]:
df = df[df["Are you self-employed?"] != 1]
df = df[(df['What is your age?'] > 18) & (df['What is your age?'] < 100)]
df = df[(df["Are you self-employed?"] != 1) & (
    df["Is your primary role within your company related to tech/IT?"].isin(['', 1, np.nan]))]
replace_dict = {"nonbinary": 1, 'female': 2, 'Female': 2, 'F': 2, 'f': 0, 'M': 0, 'm': 0, 'Male': 0, 'male': 0,
                "woman": 2, "cis male": 0, "Cis male": 0, "dude": 0}
df["What is your gender?"] = df["What is your gender?"].replace(replace_dict, regex=True)
df["What is your gender?"] = df["What is your gender?"].replace(to_replace=r'[^0-2]', value=1, regex=True)
df.replace('Not eligible for coverage / N/A', np.nan, inplace=True)
df.replace('Not applicable to me', np.nan, inplace=True)
df.replace('1-5', 0, inplace=True)
df.replace('6-25', 1, inplace=True)
df.replace('26-100', 2, inplace=True)
df.replace('100-500', 3, inplace=True)
df.replace('500-1000', 4, inplace=True)
df.replace('More than 1000', 5, inplace=True)
df['Do you work remotely?'].replace("Always", 1, inplace=True)
df['Do you work remotely?'].replace("Sometimes", 0.5, inplace=True)
df['Do you work remotely?'].replace("Never", 0, inplace=True)

df.replace("Yes", 1, inplace=True)
df.replace('No', 0, inplace= True)
df.replace('Maybe', 0.5, inplace = True)
df.replace("I don't know", 0.5, inplace = True)
df.replace("I am not sure", 0.5, inplace = True)

df.replace("Some did", 0.5, inplace=True)
df.replace("Some of them", 0.5, inplace=True)
df.replace("No, none did", 0, inplace=True)
df.replace("None of them", 0, inplace=True)
df.replace("None did", 0, inplace=True)
df.replace("Yes, they all did", 1, inplace=True)
df.replace("N/A (not currently aware)", np.nan, inplace=True)
df.replace("I was aware of some", 0.5, inplace=True)
df.replace("No, I only became aware later", 0, inplace=True)
df.replace("Yes, I was aware of all of them", 1, inplace=True)
df.replace("Yes, all of them", 1, inplace=True)
df.replace("Yes, at all of my previous employers", 1, inplace=True)
df.replace("No, at none of my previous employers", 0, inplace=True)
df.replace("Some of my previous employers", 0.5, inplace=True)

df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(0.5, 3,inplace=True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(
    "Yes, I think it would", 4, inplace=True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(
    "Yes, it has", 5, inplace=True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(
    "No, I don't think it would", 2, inplace=True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(
    "No, it has not", 1, inplace=True)
df[
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(
    0.5, 3, inplace=True)
df[
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(
    "Yes, I think they would", 4, inplace=True)
df[
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(
    "Yes, they do", 5, inplace=True)
df[
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(
    "No, I don't think they would", 2, inplace=True)
df[
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(
    "No, they do not", 1, inplace=True)

df.replace('Rarely', 1, inplace = True)
df.replace('Never', 0, inplace = True)
df.replace('Sometimes', 2, inplace = True)
df.replace('Often', 3, inplace = True)

In [4]:
df[['Does your employer provide mental health benefits as part of healthcare coverage?',
        'Do you know the options for mental health care available under your employer-provided coverage?',
        'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
        'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
        'Have your previous employers provided mental health benefits?',
        'Were you aware of the options for mental health care provided by your previous employers?',
        'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
        'Did your previous employers provide resources to learn more about mental health issues and how to seek help?']] = df[['Does your employer provide mental health benefits as part of healthcare coverage?',
        'Do you know the options for mental health care available under your employer-provided coverage?',
        'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
        'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
        'Have your previous employers provided mental health benefits?',
        'Were you aware of the options for mental health care provided by your previous employers?',
        'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
        'Did your previous employers provide resources to learn more about mental health issues and how to seek help?']].replace(np.nan, -1)

df['Employer_Resource_Avail'] = (df[['Does your employer provide mental health benefits as part of healthcare coverage?',
                                    'Do you know the options for mental health care available under your employer-provided coverage?',
                                    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
                                    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?']].max(axis = 1) + df[['Have your previous employers provided mental health benefits?',
                                    'Were you aware of the options for mental health care provided by your previous employers?',
                                    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
                                    'Did your previous employers provide resources to learn more about mental health issues and how to seek help?']].max(axis = 1)) / 2

df['Negative_Conseq'] = df[['Do you think that discussing a mental health disorder with your employer would have negative consequences?', 
                            'Do you think that discussing a mental health disorder with previous employers would have negative consequences?']].max(axis = 1)

df['Comfortable_Discussing'] = df[['Would you feel comfortable discussing a mental health disorder with your coworkers?',
                                    'Would you have been willing to discuss a mental health issue with your previous co-workers?',
                                    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
                                    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?']].mean(axis = 1)

df['Hurt_Career'] = df[['Do you feel that being identified as a person with a mental health issue would hurt your career?',
                        'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?']].max(axis = 1)

df[['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
                         'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?']] = df[['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
                         'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?']].replace(np.nan, -1)

df['Productivity'] = df[['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
                         'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?']].max(axis = 1)

df['How willing would you be to share with friends and family that you have a mental illness?'].replace(
        "Somewhat open", 4, inplace=True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Very open",
                                                                                                        5,
                                                                                                        inplace=True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace(
    "Somewhat not open", 2, inplace=True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace(
    "Not open at all", 1, inplace=True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Neutral",
                                                                                                        3,
                                                                                                        inplace=True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace(
    "Not applicable to me (I do not have a mental illness)", 0, inplace=True)

df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']] = df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']].replace(np.nan, -1)

df.replace('\.*mood.*', 2, regex=True, inplace=True)
df.replace('\.*anxiety.*', 1, regex=True, inplace=True)
df.replace('\.*Anxiety.*', 1, regex=True, inplace=True)
df.replace('\.*Mood.*', 2, regex=True, inplace=True)

df["What is your gender?"].replace(np.nan, 1, inplace=True)

df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']] = df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']].replace('\.*', -1, regex=True)

df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']] = df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?']].replace("[^0-9]", -1, regex=True)

df['Label'] = df[
    ['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?',
        'If maybe, what condition(s) do you believe you have?']].max(axis=1)

In [5]:
df = df[['What is your age?', 'What is your gender?', 'How many employees does your company or organization have?',
    'Do you work remotely?', 'Label', 'Do you have a family history of mental illness?',
    'Employer_Resource_Avail', 'Negative_Conseq', 'Comfortable_Discussing', 'Hurt_Career', 
    'How willing would you be to share with friends and family that you have a mental illness?', 'Productivity']]

In [6]:
df.rename(columns = {'What is your age?': 'age', 'What is your gender?': 'gender', 'How many employees does your company or organization have?': "num_employees",
                     'Do you work remotely?': 'work_remote', 'Do you have a family history of mental illness?': 'family_history',
                     'How willing would you be to share with friends and family that you have a mental illness?': 'share_family'}, inplace = True)

In [7]:
df.head(10)

,age,gender,num_employees,work_remote,Label,family_history,Employer_Resource_Avail,Negative_Conseq,Comfortable_Discussing,Hurt_Career,share_family,Productivity
0,39,0.0,2,0.5,1,0.0,0.25,0.5,0.625,3,4,-1.0
1,29,0.0,1,0.0,1,1.0,1.00,0.0,0.500,2,4,2.0
2,38,0.0,1,1.0,-1,0.0,0.25,0.5,0.500,3,4,-1.0
4,43,2.0,1,0.5,1,1.0,0.75,1.0,0.250,4,4,2.0
5,42,0.0,5,0.5,1,0.0,1.00,1.0,0.375,4,4,3.0
6,30,0.0,2,0.5,-1,0.0,0.50,0.0,0.750,4,0,-1.0
7,37,2.0,5,1.0,1,1.0,0.75,0.5,0.625,3,4,3.0
8,44,2.0,2,0.5,2,1.0,0.50,1.0,0.375,5,4,3.0
10,28,0.0,2,0.0,2,0.5,0.50,0.5,0.500,3,5,3.0
11,34,0.0,3,0.5,1,1.0,0.50,1.0,0.000,4,1,1.0


In [8]:
df.isna().sum()

age                        0
gender                     0
num_employees              0
work_remote                0
Label                      0
family_history             0
Employer_Resource_Avail    0
Negative_Conseq            0
Comfortable_Discussing     0
Hurt_Career                0
share_family               0
Productivity               0
dtype: int64

In [9]:
df.Label.value_counts()

-1    447
 1    410
 2    271
Name: Label, dtype: int64

In [10]:
df = df[df["Are you self-employed?"] != 1]
df = df[(df["Are you self-employed?"] != 1) & (df["Is your primary role within your company related to tech/IT?"].isin(['', 1, np.nan]))]
replace_dict = {"nonbinary":1,'female': 2, 'Female': 2, 'F':2, 'f':0,'M': 0, 'm':0, 'Male':0, 'male':0,"woman":2,"cis male":0, "Cis male":0, "dude":0}
df["What is your gender?"] = df["What is your gender?"].replace(replace_dict, regex=True)
df["What is your gender?"] = df["What is your gender?"].replace(to_replace=r'[^0-2]', value=1, regex=True)

KeyError: 'Are you self-employed?'

In [ ]:
df['What country do you live in?'].value_counts()

What country do you live in?
United States of America    695
United Kingdom              134
Canada                       63
Germany                      43
Netherlands                  31
Australia                    26
Sweden                       14
France                       11
Ireland                      10
Brazil                        9
Russia                        9
India                         7
Bulgaria                      6
New Zealand                   6
Switzerland                   6
Italy                         5
Finland                       5
Denmark                       5
South Africa                  4
Chile                         3
Norway                        3
Spain                         3
Belgium                       3
Pakistan                      3
Israel                        2
Bosnia and Herzegovina        2
Poland                        2
Romania                       2
Afghanistan                   2
Estonia                       2
Colombia   

In [ ]:
df.replace('Not eligible for coverage / N/A', np.nan, inplace = True)
df.replace('Not applicable to me', np.nan, inplace = True)
df.replace('I am not sure', 'Unsure', inplace = True)
df.replace("I don't know", 'Unsure', inplace = True)
df.replace('Very difficult', 5, inplace = True)
df.replace('Somewhat difficult', 4, inplace = True)
df.replace('Neither easy nor difficult', 3, inplace=True)
df.replace('Somewhat easy', 2, inplace = True)
df.replace('Very easy', 1, inplace = True)
df.replace("Yes", 1, inplace = True)
df.replace("Yes, always", 1, inplace = True)
df.replace("Maybe", 0.5, inplace=True)
df.replace("Sometimes", 0.5, inplace=True)
df.replace("No", 0, inplace=True)
df['Do you feel that your employer takes mental health as seriously as physical health?'].replace("Unsure", 0.5, inplace = True)
df.replace("Some did", 0.5, inplace = True)
df.replace("Some of them", 0.5, inplace = True)
df.replace("No, none did", 0, inplace = True)
df.replace("None of them", 0, inplace = True)
df.replace("None did", 0, inplace = True)
df.replace("Yes, they all did", 1, inplace = True)
df.replace("N/A (not currently aware)", np.nan, inplace = True)
df.replace("I was aware of some", 0.5, inplace = True)
df.replace("No, I only became aware later", 0, inplace = True)
df.replace("Yes, I was aware of all of them", 1, inplace = True)
df.replace("Yes, all of them", 1, inplace = True)
df.replace("Yes, at all of my previous employers", 1, inplace = True)
df.replace("No, at none of my previous employers", 0, inplace = True)
df.replace("Some of my previous employers", 0.5, inplace = True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace(0.5, 3, inplace = True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace("Yes, I think it would", 4, inplace = True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace("Yes, it has", 5, inplace = True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace("No, I don't think it would", 2, inplace = True)
df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].replace("No, it has not", 1, inplace = True)
df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace(0.5, 3, inplace = True)
df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace("Yes, I think they would", 4, inplace = True)
df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace("Yes, they do", 5, inplace = True)
df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace("No, I don't think they would", 2, inplace = True)
df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].replace("No, they do not", 1, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Somewhat open", 4, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Very open", 5, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Somewhat not open", 2, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Not open at all", 1, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Neutral", 3, inplace = True)
df['How willing would you be to share with friends and family that you have a mental illness?'].replace("Not applicable to me (I do not have a mental illness)", np.nan, inplace = True)
df.replace("Maybe/Not sure", 0.5, inplace = True)
df.replace("Yes, I observed", 1, inplace = True)
df.replace("Yes, I experienced", 2, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].replace(0.5, 2, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].replace("Rarely", 1, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].replace("Often", 3, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].replace("Never", 0, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].replace(0.5, 2, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].replace("Rarely", 1, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].replace("Often", 3, inplace = True)
df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].replace("Never", 0, inplace = True)
df['Do you work remotely?'].replace("Always", 1, inplace = True)
df['Do you work remotely?'].replace("Never", 0, inplace = True)
df['Do you work remotely?'].replace("Never", 0, inplace = True)
df.replace('1-5', 0, inplace = True)
df.replace('6-25', 1, inplace = True)
df.replace('26-100', 2, inplace = True)
df.replace('100-500', 3, inplace = True)
df.replace('500-1000', 4, inplace = True)
df.replace('More than 1000', 5, inplace = True)
df['Does your employer provide mental health benefits as part of healthcare coverage?'].replace(np.NaN, 0, inplace=True)
df['Do you know the options for mental health care available under your employer-provided coverage?'].replace('Unsure', 0.5, inplace=True)
df['Do you know the options for mental health care available under your employer-provided coverage?'].replace(np.NaN, 0.25, inplace=True)
df['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'].replace('Unsure', 0.5, inplace=True)
df['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'].replace(np.NaN, 0.5, inplace=True)
df.replace(np.NaN, -1.0, inplace=True)

df.replace('\.*mood.*', 2, regex=True, inplace=True)
df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?', 'If maybe, what condition(s) do you believe you have?']].replace('\.*anxiety.*', 1, regex=True, inplace=True)
df.replace('\.*Anxiety.*', 1, regex=True, inplace=True)
df.replace('\.*Mood.*', 2, regex=True, inplace=True)

df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?', 'If maybe, what condition(s) do you believe you have?']].replace('\.*', 3, regex=True, inplace=True)

df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?', 'If maybe, what condition(s) do you believe you have?']].replace("[^0-9]", 3, regex=True, inplace=True)

df.replace("[^0-9]", -1.0, regex=True, inplace=True)

df['Label'] = df[['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?', 'If maybe, what condition(s) do you believe you have?']].max(axis=1)

df.drop(['If so, what condition(s) were you diagnosed with?', 'If yes, what condition(s) have you been diagnosed with?', 'If maybe, what condition(s) do you believe you have?'], axis=1,inplace=True)

C:\Users\ansel\AppData\Local\Temp\ipykernel_18292\3967767804.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('Not eligible for coverage / N/A', np.nan, inplace = True)
C:\Users\ansel\AppData\Local\Temp\ipykernel_18292\3967767804.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("No", 0, inplace=True)
C:\Users\ansel\AppData\Local\Temp\ipykernel_18292\3967767804.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The beh

In [ ]:
df = df[(df['What is your age?'] > 18) & (df['What is your age?'] < 100)]


In [ ]:


# print(df.columns)
df['Do you think that discussing mental health at work would have negative consequences?'] = df[['Do you think that discussing a mental health disorder with your employer would have negative consequences?', 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:', 'Do you think that discussing a physical health issue with your employer would have negative consequences?', 'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?', 'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?']].sum(axis=1)

df['Comfort Discussing Mental Health problems'] = df[[ 'Would you feel comfortable discussing a mental health disorder with your coworkers?','Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?','Do you feel that your employer takes mental health as seriously as physical health?', 'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?', 'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?']].sum(axis=1)


df.drop(['Are you self-employed?', 'Is your employer primarily a tech company/organization?', 'Is your primary role within your company related to tech/IT?', 'Why or why not?', 'Why or why not?.1', 'What country do you work in?', 'What country do you live in?', 'What US state or territory do you live in?', 'What US state or territory do you work in?', 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'], inplace=True,axis=1)

df.drop(['Would you feel comfortable discussing a mental health disorder with your coworkers?','Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?','Do you feel that your employer takes mental health as seriously as physical health?', 'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?', 'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?'], inplace=True, axis=1)

df.drop(['Do you think that discussing a mental health disorder with your employer would have negative consequences?', 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:', 'Do you think that discussing a physical health issue with your employer would have negative consequences?', 'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?', 'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?'], inplace=True, axis=1)

print(df.columns)
df.head()

Index(['How many employees does your company or organization have?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided coverage?',
       'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'Do you have medical coverage (private insurance or state-provided) which includes treatment of  mental health issues?',
       'Do you know local or online resources to seek help for a mental health disorder?',
       'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?',
       'Do you believe your productivity is ever affected by a mental health issue?',
       'If yes, what p

,How many employees does your company or organization have?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",...,Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,Which of the following best describes your work position?,Do you work remotely?,Label,Do you think that discussing mental health at work would have negative consequences?,Comfort Discussing Mental Health problems
0,2,0.0,0.25,0.0,0.5,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,-1.0,-1.0,39,0.0,-1.0,0.5,1.0,0.0,0.0
1,1,0.0,1.00,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,1.0,2.0,29,0.0,-1.0,0.0,1.0,1.0,0.5
2,1,0.0,0.25,0.0,0.5,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,-1.0,-1.0,38,0.0,-1.0,1.0,-1.0,2.5,-0.5
4,1,1.0,1.00,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,2.0,2.0,43,2.0,-1.0,0.5,1.0,3.5,-1.5
5,5,1.0,0.50,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1,-1.0,3.0,42,0.0,-1.0,0.5,1.0,4.0,-0.5


<Figure size 1600x400 with 0 Axes>

In [ ]:

print(df['Label'].value_counts())

y = df['Label']
X = df.drop(['Label'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)
clf = RandomForestClassifier()
print(cross_val_score(clf,X,y,cv=3))

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

print(X_test.columns)

Label
-1.0    448
 1.0    409
 2.0    271
Name: count, dtype: int64
[0.71808511 0.6356383  0.66223404]
              precision    recall  f1-score   support

        -1.0       0.96      0.83      0.89       156
         1.0       0.53      0.79      0.64       131
         2.0       0.33      0.16      0.22        86

    accuracy                           0.66       373
   macro avg       0.60      0.59      0.58       373
weighted avg       0.66      0.66      0.64       373

Index(['How many employees does your company or organization have?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided coverage?',
       'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provide

In [ ]:
print(clf.predict(X_test))


[ 1.  1.  1.  1.  1.  1.  2.  1.  1. -1. -1.  1.  1.  2.  1. -1.  2.  1.
  2. -1.  1.  2.  1.  1.  1. -1.  1.  2.  1.  1.  1.  1. -1. -1. -1.  1.
  2. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1.
 -1.  1. -1.  1. -1. -1.  1.  1.  2.  1.  1. -1.  1. -1.  2. -1.  1. -1.
 -1.  1.  1.  1.  2.  1.  1.  2.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1.  1. -1. -1.  2.  1.  1.  1.  1. -1.  1. -1. -1.  1.  2. -1. -1.  1.
 -1. -1.  1.  2.  1. -1. -1. -1.  1. -1.  1. -1. -1.  1. -1.  1.  1. -1.
 -1.  1.  1.  1.  1.  2.  1.  1.  1.  1.  2. -1.  2. -1.  2. -1.  1. -1.
  1.  2. -1.  1. -1.  2.  2.  1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1.
  1. -1.  1.  1. -1. -1.  1. -1.  2. -1.  1.  1.  1.  2.  1.  1. -1.  1.
 -1.  1. -1. -1.  1. -1. -1.  1.  2.  1. -1.  1. -1.  1. -1.  1. -1. -1.
  1.  1. -1. -1.  1.  1. -1.  2.  1. -1.  2.  2.  1. -1. -1.  1. -1.  2.
  1. -1.  2. -1.  1.  2.  1.  2.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1.  1. -1.  2. -1. -1. -1.  1.  1.  1.  1